
by: Hamid Reza Mazandarani @ Part AI Research Center

In this notebook, we use tsfresh library to extract features from bitcoin price time series

In [3]:
import numpy as np
import pandas as pd

# import your dataset

df = df[df.date >= '2021-01-01']

df.reset_index(inplace=True, drop=True)

df


,date,open,high,low,close,volume,tic
0,2021-01-01,28951.7,29627.1,28712.4,29359.9,100902.0,BTC-USD
1,2021-01-02,29359.7,33233.5,29008.0,32193.3,240865.0,BTC-USD
2,2021-01-03,32192.9,34755.9,32029.6,32958.9,155210.0,BTC-USD
3,2021-01-04,33016.6,33587.5,28204.5,32022.6,255269.0,BTC-USD
4,2021-01-05,32015.4,34414.7,30010.5,33991.5,202128.0,BTC-USD
...,...,...,...,...,...,...,...
246,2021-09-04,49999.3,50534.6,49461.4,49918.4,48142.0,BTC-USD
247,2021-09-05,49918.7,51878.2,49492.4,51768.6,57301.0,BTC-USD
248,2021-09-06,51769.4,52726.2,51002.8,52672.1,69456.0,BTC-USD
249,2021-09-07,52674.2,52885.3,43072.4,46779.6,187558.0,BTC-USD


In [4]:
from tsfresh.utilities.dataframe_functions import roll_time_series

df_rolled = roll_time_series(df, column_id="tic", column_sort="date")

df_rolled = df_rolled.drop(['tic'], axis=1)

df_rolled


Rolling: 100%|██████████| 10/10 [00:00<00:00, 28.27it/s]


,date,open,high,low,close,volume,id
0,2021-01-01,28951.7,29627.1,28712.4,29359.9,100902.0,"(BTC-USD, 2021-01-01)"
1,2021-01-01,28951.7,29627.1,28712.4,29359.9,100902.0,"(BTC-USD, 2021-01-02)"
2,2021-01-02,29359.7,33233.5,29008.0,32193.3,240865.0,"(BTC-USD, 2021-01-02)"
3,2021-01-01,28951.7,29627.1,28712.4,29359.9,100902.0,"(BTC-USD, 2021-01-03)"
4,2021-01-02,29359.7,33233.5,29008.0,32193.3,240865.0,"(BTC-USD, 2021-01-03)"
...,...,...,...,...,...,...,...
31621,2021-09-04,49999.3,50534.6,49461.4,49918.4,48142.0,"(BTC-USD, 2021-09-08)"
31622,2021-09-05,49918.7,51878.2,49492.4,51768.6,57301.0,"(BTC-USD, 2021-09-08)"
31623,2021-09-06,51769.4,52726.2,51002.8,52672.1,69456.0,"(BTC-USD, 2021-09-08)"
31624,2021-09-07,52674.2,52885.3,43072.4,46779.6,187558.0,"(BTC-USD, 2021-09-08)"


In [5]:
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters

extracted_features = extract_features(df_rolled, column_id="id", column_sort="date")


Feature Extraction: 100%|██████████| 10/10 [00:49<00:00,  4.93s/it]


In [6]:
from tsfresh.utilities.dataframe_functions import impute

extracted_features = impute(extracted_features)


/home/hamidrezamazandarani/Desktop/Workspace/myProject/venv/lib/python3.8/site-packages/tsfresh/utilities/dataframe_functions.py:171: RuntimeWarning: The columns ['open__query_similarity_count__query_None__threshold_0.0'
 'high__query_similarity_count__query_None__threshold_0.0'
 'low__query_similarity_count__query_None__threshold_0.0'
 'close__query_similarity_count__query_None__threshold_0.0'
 'volume__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  warnings.warn("The columns {} did not have any finite values. Filling with zeros.".format(


In [7]:
from tsfresh import select_features

trend_type = lambda horizon: df['close'].rolling(horizon).mean().fillna(0)
target = (trend_type(4).shift(-4) / trend_type(3))
target.replace([np.inf, -np.inf], np.nan, inplace=True)
target = target.fillna(method="bfill").fillna(method="ffill")
target.index = extracted_features.index

small_feature_set = select_features(extracted_features, target, fdr_level = 0.00001)

In [8]:
small_feature_set

close__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4  \
BTC-USD 2021-01-01                                           0.000000                   
        2021-01-02                                           0.000000                   
        2021-01-03                                           0.000000                   
        2021-01-04                                           0.000000                   
        2021-01-05                                         765.600000                   
...                                                               ...                   
        2021-09-04                                        1317.990244                   
        2021-09-05                                        1324.402410                   
        2021-09-06                                        1319.391667                   
        2021-09-07                                        1373.192941                   
        2021-09-08                                        1365.576744                   

                    open__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4  \
BTC-USD 2021-01-01                                           0.000000                  
        2021-01-02                                           0.000000                  
        2021-01-03                                           0.000000                  
        2021-01-04                                           0.000000                  
        2021-01-05                                           0.000000                  
...                                                               ...                  
        2021-09-04                                        1323.545122                  
        2021-09-05                                        1308.569880                  
        2021-09-06                                        1322.404819                  
        2021-09-07                                        1317.433333                  
        2021-09-08                                        1371.347059                  

                    close__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2  \
BTC-USD 2021-01-01                                           0.000000                   
        2021-01-02                                           0.000000                   
        2021-01-03                                           0.000000                   
        2021-01-04                                           0.000000                   
        2021-01-05                                         850.950000                   
...                                                               ...                   
        2021-09-04                                        1449.475758                   
        2021-09-05                                        1452.488722                   
        2021-09-06                                        1448.391791                   
        2021-09-07                                        1481.311111                   
        2021-09-08                                        1475.700000                   

                    open__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0  \
BTC-USD 2021-01-01                                           0.000000                  
        2021-01-02                                           0.000000                  
        2021-01-03                                         408.000000                  
        2021-01-04                                        1620.600000                  
        2021-01-05                                        1620.600000                  
...                                                               ...                  
        2021-09-04                                        1396.841053                  
        2021-09-05                                        1389.949738                  
        2021-09-06               

In [9]:
small_feature_set.columns

Index(['close__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4',
       'open__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4',
       'close__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',
       'open__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',
       'open__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0',
       'open__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',
       'close__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0',
       'volume__change_quantiles__f_agg_"mean"__isabs_False__qh_1.0__ql_0.6',
       'close__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',
       'close__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.4',
       'open__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.4',
       'open__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2',
       'high__change_quantiles__f_agg_"var"__isabs_True__qh_0.4__ql_0.0',
       'open__chan